In [5]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import FeatureHasher

In [6]:
# download the data here
# !curl -L -o ../data/sample5.csv "https://drive.google.com/uc?export=download&id=1KEMfwDKxd5klTA9tFo6Nsgmi0pANIl7m"

In [7]:
url ='https://drive.google.com/uc?id=1KEMfwDKxd5klTA9tFo6Nsgmi0pANIl7m'
df = pd.read_csv(url)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          50000 non-null  int64  
 1   make          49047 non-null  object 
 2   model         49037 non-null  object 
 3   trim          49012 non-null  object 
 4   body          48782 non-null  object 
 5   transmission  44014 non-null  object 
 6   vin           49999 non-null  object 
 7   state         50000 non-null  object 
 8   condition     48923 non-null  float64
 9   odometer      49995 non-null  float64
 10  color         49946 non-null  object 
 11  interior      49946 non-null  object 
 12  seller        50000 non-null  object 
 13  sellingprice  50000 non-null  int64  
 14  saledate      50000 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 5.7+ MB


In [9]:
df.sample(5)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,sellingprice,saledate
31342,2014,Kia,Forte,LX,Sedan,NaN,knafx4a69e5239926,co,47.0,10666.0,red,gray,kia motors america inc,12600,Tue Feb 17 2015 04:00:00 GMT-0800 (PST)
49009,2013,Kia,Forte,EX,sedan,automatic,knafu4a21d5740443,ca,32.0,57238.0,red,gray,the hertz corporation,8800,Tue Jun 09 2015 05:30:00 GMT-0700 (PDT)
623,2012,Hyundai,Elantra,GLS,Sedan,NaN,5npdh4ae6ch125443,pa,4.0,26678.0,silver,gray,hyundai motor finance,11300,Fri Jan 02 2015 09:00:00 GMT-0800 (PST)
29396,2004,Buick,LeSabre,Custom,Sedan,automatic,1g4hp54k344156177,nj,27.0,50351.0,beige,tan,lester glenn buick,4200,Wed Feb 04 2015 01:30:00 GMT-0800 (PST)
40934,2008,Dodge,Ram Pickup 2500,SLT,quad cab,automatic,3d7ks28a88g116359,pa,46.0,74220.0,blue,gray,my car llc,28800,Fri May 29 2015 02:30:00 GMT-0700 (PDT)


In [10]:
# Before encoding, take a look at the unique values
for col in df.columns:
  print(col, df[col].nunique())

year 27
make 80
model 751
trim 1222
body 73
transmission 3
vin 49925
state 40
condition 41
odometer 41966
color 21
interior 17
seller 5350
sellingprice 971
saledate 2321


In [11]:
for col in df.columns:
  print(col, df[col].unique())

year [2013 2009 2008 2014 2012 2006 2003 2007 2015 2005 2002 2004 2001 2000
 1999 2011 2010 1994 1996 1998 1993 1997 1995 1992 1991 1990 1986]
make ['Ford' 'Infiniti' 'Jeep' 'Kia' 'Toyota' 'Nissan' 'Chrysler' 'BMW'
 'Chevrolet' 'Porsche' nan 'GMC' 'Hyundai' 'Buick' 'Lincoln' 'Cadillac'
 'Lexus' 'Honda' 'Acura' 'chrysler' 'Pontiac' 'Audi' 'Mercedes-Benz'
 'Volkswagen' 'Dodge' 'Ram' 'Mazda' 'HUMMER' 'Jaguar' 'Subaru' 'MINI'
 'Mitsubishi' 'Volvo' 'Land Rover' 'Mercury' 'land rover' 'Scion'
 'lincoln' 'Fisker' 'Bentley' 'ford' 'Saturn' 'jeep' 'Lamborghini' 'FIAT'
 'Saab' 'smart' 'Maserati' 'chevrolet' 'Suzuki' 'toyota' 'pontiac'
 'nissan' 'Oldsmobile' 'mazda' 'honda' 'lexus' 'cadillac' 'dodge'
 'plymouth' 'vw' 'Ferrari' 'mitsubishi' 'Isuzu' 'bmw' 'subaru' 'Geo' 'gmc'
 'mercedes' 'Aston Martin' 'oldsmobile' 'mercury' 'volkswagen' 'gmc truck'
 'maserati' 'acura' 'Tesla' 'landrover' 'hyundai' 'Rolls-Royce' 'porsche']
model ['Fusion' 'JX' 'Grand Cherokee' 'F-350 Super Duty' 'Sportage' 'Tacoma'

## Clean the data

### Clean up `vin`

In [12]:
vin_up = df['vin'].astype(str).str.upper()

pattern = r'^[A-HJ-NPR-Z0-9]{17}$'

valid_vin_mask = vin_up.str.match(pattern)

df_valid = df.loc[valid_vin_mask].copy()

dropped = (~valid_vin_mask).sum()
print(f"{dropped=}")

dropped=2


### Clean up `make`

In [13]:
make_map = {
    'vw':'Volkswagen',
    'volkswagen':'Volkswagen',
    'gmc':'GMC',
    'gmc truck':'GMC',
    'chevy':'Chevrolet',
    'chevrolet':'Chevrolet',
    'ford':'Ford',
    'bwm':'BMW',
    'bmw':'BMW',
    'hummer':'HUMMER',
    'land rover':'Land Rover',
    'landrover':'Land Rover',
    'mercedes':'Mercedes-Benz',
    'mercedes-benz':'Mercedes-Benz'
}

In [14]:
s = df['make'].str.strip().str.lower()
df['make'] = s.map(make_map)

In [15]:
mask = df['make'].isna() & s.notna()
df.loc[mask, 'make'] = s[mask].str.title()

### Clean up `body`

In [16]:
body_map = {
    'sedan':'Sedan',
    'suv':'SUV',
    'crew cab':'Crew Cab',
    'double cab':'Double Cab',
    'regular cab':'Regular Cab',
    'quad cab':'Quad Cab',
    'extended cab':'Extended Cab',
    'king cab':'King Cab',
    'crewmax cab':'CrewMax Cab',
    'supercab':'SuperCab',
    'supercrew':'SuperCrew',
    'hatchback':'Hatchback',
    'coupe':'Coupe',
    'convertible':'Convertible',
    'wagon':'Wagon',
    'minivan':'Minivan',
    'van':'Van',
    'e-series van':'E-Series Van',
    'access cab':'Access Cab',
    'mega cab':'Mega Cab',
    'transit van':'Transit Van',
    'promaster cargo van':'Promaster Cargo Van',
    'xtracab':'Xtracab',
    'g sedan':'Sedan',
    'g coupe':'Coupe',
    'g convertible':'Convertible',
    'genesis coupe':'Coupe',
    'elantra coupe':'Coupe',
    'tsx sport wagon':'Wagon',
    'beetle convertible':'Convertible',
    'cts wagon':'Wagon',
    'cts coupe':'Coupe',
    'cts-v coupe':'Coupe',
    'q60 convertible':'Convertible',
    'q60 coupe':'Coupe',
    'g37 convertible':'Convertible',
    'navitgation':np.nan,
    'cab plus 4':np.nan,
}

s = df['body'].astype(str).str.strip().str.lower()

df['body'] = s.map(body_map)

mask = df['body'].isna() & s.notna()
df.loc[mask, 'body'] = s[mask].str.replace('-', ' ').str.title()

df['body'] = df['body'].replace({'': np.nan, 'nan': np.nan})

### Clean up `transmission`

In [17]:
trans_map = {
    'automatic': 'Automatic',
    'manual':    'Manual',
}

s = df['transmission'].astype(str).str.strip().str.lower()

df['transmission'] = s.map(trans_map)

### Clean up `state`

In [18]:
import re

st = df['state'].str.upper()

valid = st.str.match(r'^[A-Z]{2}$')

df.loc[~valid, 'state'] = np.nan

### Clean up `color`

In [19]:
for col in ['color','interior']:
    s = df[col].astype(str).str.strip()
    
    s = s.replace({'—': np.nan})

    is_digits = s.str.fullmatch(r'\d+').fillna(False)

    s = s.where(~is_digits, np.nan)

    df[col] = (
        s
        .str.lower()
        .str.replace('-', ' ')
        .str.title()
        .replace({'Nan': np.nan})
    )

/tmp/ipykernel_879/2392785884.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_digits = s.str.fullmatch(r'\d+').fillna(False)
/tmp/ipykernel_879/2392785884.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_digits = s.str.fullmatch(r'\d+').fillna(False)


### Convert `saledate` to datetime

In [20]:
import pandas as pd

weekday_pattern = r'^(Mon|Tue|Wed|Thu|Fri|Sat|Sun)\s'
mask_dates = df['saledate'].str.match(weekday_pattern, na=False)
df.loc[~mask_dates, 'saledate'] = np.nan

clean = df['saledate'].str.replace(r'\s*\(.*\)$', '', regex=True)

df['saledate'] = pd.to_datetime(
    clean,
    format="%a %b %d %Y %H:%M:%S GMT%z",
    utc=True,
    errors='coerce'
)

In [21]:
df['weekday'] = df['saledate'].dt.day_name()

## Split

In [22]:
df.sample(5)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,sellingprice,saledate,weekday
8749,2012,Ford,Expedition,EL King Ranch,SUV,Automatic,1fmjk1j58cef02224,fl,28.0,60718.0,Black,Gray,tdaf remarketing,23400,2015-06-09 09:00:00+00:00,Tuesday
11559,2014,Ram,1500,SLT,Quad Cab,Automatic,1c6rr6gtxes324625,pa,5.0,26612.0,Red,Gray,enterprise veh exchange/rental,22300,2015-06-11 09:30:00+00:00,Thursday
19506,2013,Volkswagen,Jetta SportWagen,TDI,Wagon,Automatic,3vwpl7aj5dm615312,fl,21.0,19084.0,White,Beige,cox chevrolet inc,16200,2015-02-10 09:15:00+00:00,Tuesday
5469,2001,Ford,Explorer Sport,Base,SUV,Automatic,1fmyu70e61ub41981,ny,NaN,146528.0,Red,Gray,fuccillo kia,650,2014-12-18 17:15:00+00:00,Thursday
21187,2011,Nissan,Sentra,2.0 SR,Sedan,Automatic,3n1ab6ap1bl652678,ga,26.0,82348.0,Black,Gray,landmark auto inc,6600,2015-02-12 10:00:00+00:00,Thursday


In [23]:
X = df.drop('sellingprice', axis=1)
y = df['sellingprice']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1
)

In [25]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

## Impute missing values

In [26]:
num_cols = ['year', 'condition', 'odometer']
num_imputer = SimpleImputer(strategy='median')

In [27]:
cat_cols = ['make', 'model', 'trim', 'body', 'transmission', 'color', 'interior', 'seller', 'weekday', 'state']
cat_imputer = SimpleImputer(strategy='most_frequent')

In [28]:
X_train[num_cols] = num_imputer.fit_transform(X_train[num_cols])
X_train[cat_cols] = cat_imputer.fit_transform(X_train[cat_cols])

In [29]:
# transform without fitting on the test set
X_test[num_cols] = num_imputer.transform(X_test[num_cols])
X_test[cat_cols] = cat_imputer.transform(X_test[cat_cols])

In [30]:
median_saledate = X_train['saledate'].median()

for df in (X_train, X_test):
    df['saledate'] = df['saledate'].fillna(median_saledate)

## Deal with datetime

In [31]:
import calendar

for df in (X_train, X_test):
    df['sale_year'] = df['saledate'].dt.year
    df['sale_month'] = df['saledate'].dt.month
    df['sale_day'] = df['saledate'].dt.day
    df['sale_week'] = df['saledate'].dt.isocalendar().week.astype(int)
    df['sale_quarter'] = df['saledate'].dt.quarter
    df['sale_hour'] = df['saledate'].dt.hour
    df['dow_num'] = df['saledate'].dt.weekday
    df['is_weekend'] = (df['dow_num'] >= 5).astype(int)


X_train.drop(['saledate','weekday'], axis=1, inplace=True)
X_test.drop(['saledate','weekday'], axis=1, inplace=True)

### Use cyclical encoding to capture seasonality

In [32]:
def cyc_encode(df, col, period):
    df[f'{col}_sin'] = np.sin(2*np.pi * df[col] / period)
    df[f'{col}_cos'] = np.cos(2*np.pi * df[col] / period)

for df in (X_train, X_test):
    cyc_encode(df, 'dow_num', 7)
    cyc_encode(df, 'sale_month', 12)
    df.drop(['dow_num','sale_month'], axis=1, inplace=True)

### Turn `year` into `age`

In [33]:
for df in (X_train, X_test):
    df['age'] = df['sale_year'] - df['year']

In [34]:
X_train.drop('year', axis=1, inplace=True)
X_test.drop('year', axis=1, inplace=True)

In [35]:
age_bins   = [0, 3, 6, 10, 20, 100] # tackle problem from the giskard report
age_labels = ['0-3y','4-6y','7-10y','11-20y','20+y']
for df in (X_train, X_test):
    df['age_bin'] = pd.cut(df['age'], bins=age_bins, labels=age_labels)

In [36]:
age_encoder = OneHotEncoder(
    drop='first',
    sparse_output=False,
    handle_unknown='ignore'
)

In [37]:
age_train = age_encoder.fit_transform(X_train[['age_bin']])
age_test = age_encoder.transform(X_test [['age_bin']])

In [38]:
X_train.drop(['age','age_bin'], axis=1, inplace=True)
X_test.drop(['age','age_bin'], axis=1, inplace=True)

## Encode features

## Scaling

In [39]:
scaler = StandardScaler()

In [40]:
X_train[['condition','odometer']] = scaler.fit_transform(
    X_train[['condition','odometer']]
)
X_test[['condition','odometer']] = scaler.transform(
    X_test[['condition','odometer']]
)

### Label encoding

In [41]:
transmission_encoder = LabelEncoder()
X_train['transmission'] = transmission_encoder.fit_transform(X_train['transmission'])

X_test['transmission'] = transmission_encoder.transform(X_test['transmission'])

### One-hot

In [42]:
color_encoder = OneHotEncoder(
    drop='first',
    sparse_output=False,
    handle_unknown='ignore'
)
color_train = color_encoder.fit_transform(X_train[['color']])
color_test = color_encoder.transform(X_test[['color']])

In [43]:
interior_encoder = OneHotEncoder(
    drop='first',
    sparse_output=False,
    handle_unknown='ignore'
)
interior_train = interior_encoder.fit_transform(X_train[['interior']])
interior_test  = interior_encoder.transform(X_test[['interior']])

In [44]:
body_encoder = OneHotEncoder(
    handle_unknown='ignore',
    sparse_output=False,
    drop='first'
)

body_train = body_encoder.fit_transform(X_train[['body']])
body_test = body_encoder.transform(X_test[['body']])

In [45]:
make_encoder = OneHotEncoder(
    handle_unknown='ignore',
    sparse_output=False,
    drop='first'
)

make_train = make_encoder.fit_transform(X_train[['make']])
make_test = make_encoder.transform(X_test[['make']])

### Hashing

In [46]:
hash_configs = [
    ('state', 10),
    ('model', 50),
    ('trim',   50),
    ('seller', 100)
]

In [47]:
hashed_train_parts = []
hashed_test_parts  = []

In [48]:
for col, n_feats in hash_configs:
    hasher = FeatureHasher(n_features=n_feats, input_type='string')

    H_train = hasher.transform(X_train[col].astype(str).values.reshape(-1, 1))
    H_test  = hasher.transform(X_test [col].astype(str).values.reshape(-1, 1))

    # take absolute so we don't have negatives (from the giskard report)
    train_arr = np.abs(H_train.toarray())
    test_arr  = np.abs(H_test.toarray())

    df_train = pd.DataFrame(
        train_arr,
        columns=[f'{col}_hash_{i}' for i in range(n_feats)]
    )
    df_test = pd.DataFrame(
        test_arr,
        columns=[f'{col}_hash_{i}' for i in range(n_feats)]
    )

    hashed_train_parts.append(df_train)
    hashed_test_parts.append(df_test)

In [49]:
to_drop = ['color', 'interior', 'state', 'body', 'make', 'model', 'trim', 'seller']
base_train = X_train.drop(to_drop, axis=1)
base_test = X_test.drop(to_drop, axis=1)

## Concatenate everything

In [50]:
X_train = pd.concat(
    [ base_train,
      pd.DataFrame(color_train, columns=color_encoder.get_feature_names_out(['color'])),
      pd.DataFrame(interior_train, columns=interior_encoder.get_feature_names_out(['interior'])),
      pd.DataFrame(body_train,  columns=body_encoder.get_feature_names_out(['body'])),
      pd.DataFrame(make_train,  columns=make_encoder.get_feature_names_out(['make'])),
      pd.DataFrame(age_train, columns=age_encoder.get_feature_names_out(['age_bin'])),
      *hashed_train_parts
    ],
    axis=1
)

X_test = pd.concat(
    [ base_test,
      pd.DataFrame(color_test, columns=color_encoder.get_feature_names_out(['color'])),
      pd.DataFrame(interior_test, columns=interior_encoder.get_feature_names_out(['interior'])),
      pd.DataFrame(body_test,  columns=body_encoder.get_feature_names_out(['body'])),
      pd.DataFrame(make_test,  columns=make_encoder.get_feature_names_out(['make'])),
      pd.DataFrame(age_test,  columns=age_encoder.get_feature_names_out(['age_bin'])),
      *hashed_test_parts
    ],
    axis=1
)

In [51]:
cols_to_drop = ['vin'] # TODO: Don't drop vin, use it to get some other features

X_train = X_train.drop(cols_to_drop, axis=1)
X_test = X_test.drop(cols_to_drop, axis=1)

In [52]:
for col in X_train.columns:
  print(col, X_train[col].nunique())

transmission 2
condition 41
odometer 34684
sale_year 2
sale_day 31
sale_week 31
sale_quarter 4
sale_hour 20
is_weekend 2
dow_num_sin 7
dow_num_cos 7
sale_month_sin 7
sale_month_cos 8
color_Black 2
color_Blue 2
color_Brown 2
color_Burgundy 2
color_Charcoal 2
color_Gold 2
color_Gray 2
color_Green 2
color_Off White 2
color_Orange 2
color_Pink 2
color_Purple 2
color_Red 2
color_Silver 2
color_Turquoise 2
color_White 2
color_Yellow 2
interior_Black 2
interior_Blue 2
interior_Brown 2
interior_Burgundy 2
interior_Gold 2
interior_Gray 2
interior_Green 2
interior_Off White 2
interior_Orange 2
interior_Purple 2
interior_Red 2
interior_Silver 2
interior_Tan 2
interior_White 2
interior_Yellow 2
body_Cab Plus 4 2
body_Club Cab 2
body_Convertible 2
body_Coupe 2
body_Crew Cab 2
body_CrewMax Cab 2
body_Double Cab 2
body_E-Series Van 2
body_Extended Cab 2
body_Granturismo Convertible 2
body_Hatchback 2
body_King Cab 2
body_Koup 2
body_Mega Cab 2
body_Minivan 2
body_Nan 2
body_Navitgation 2
body_Promast

In [53]:
X_train.shape

(40000, 338)

## Save everything

In [55]:
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)